In [476]:
import pandas as pd
import re
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA

In [477]:
news = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')

In [478]:
news = news.dropna(subset=['content', 'source'])
news[:5]

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [479]:
news = news.sample(5000)

In [480]:
news['xinhua'] = np.where(news['source'].str.contains('新华'), 1, 0)
news[:5]

,id,author,source,content,feature,title,url,xinhua
13411,76201,倪元锦,新华社,新华社北京４月３日电记者３日下午从北京市环保局获悉，３日１５时起北京市空气重污染黄色预警...,"{""type"":""天气环保"",""site"":""新华社"",""url"":""http://home...",（新华简讯）京津冀空气污染加重 北京升级为“橙色预警”,http://home.xinhua-news.com/gdsdetailxhsnew/17...,1
76426,13186,zhanglixin,新华社,新华社台北5月26日电（记者章利新马玉洁）台湾女作家林奕含自杀案引发岛内各界挞伐“狼师”，呼...,"{""type"":""政治法律"",""site"":""新华社"",""url"":""http://home...",（港澳台）增订“狼师条款” 台湾规定补习班实行实名制,http://home.xinhua-news.com/gdsdetailxhsnew/21...,1
75666,13946,美联,新华社,新华社照片，美联，2017年5月26日\n（体育）（6）篮球——NBA东部决赛：骑士队获东部...,"{""type"":""体育"",""site"":""新华社"",""url"":""http://home.x...",（体育）（6）篮球——NBA东部决赛：骑士队获东部冠军,http://home.xinhua-news.com/gdsdetailxhsnew/21...,1
9867,79750,周扬,环球时报,资料图\r\n 【环球时报综合报道】“我从未向美国寻求帮助”，菲律宾总统杜特尔特11日否认...,"{""type"":""军事"",""site"":""环球"",""commentNum"":""2"",""joi...",美特种兵介入菲反恐战事 杜特尔特却否认求援,http://mil.huanqiu.com/world/2017-06/10821486....,0
4613,85004,NaN,中国新闻网,中新网南宁6月23日电(记者 林艳华)为深入实施高校招生阳光工程，进一步推动考试招生工作公开...,"{""type"":""社会滚动供wifi"",""site"":""参考消息"",""commentNum""...",6月23日至24日广西考生可申请高考统考成绩复核,http://www.cankaoxiaoxi.com/society/rollwifi/2...,0


In [481]:
y = news['xinhua'].tolist()
y = np.array(y)

In [482]:
np.where(news['source'].str.contains('新华'), 1, 0)

array([1, 1, 1, ..., 0, 1, 1])

In [483]:
len(news[news['xinhua']==0])

460

In [484]:
sample = news['content'].tolist()

In [485]:
a = ''.join(re.findall(re.compile('\w|\d'), sample[0]))
b = ' '.join(jieba.cut(a))
b

'新华社 北京 ４ 月 ３ 日电 记者 ３ 日 下午 从 北京市 环保局 获悉 ３ 日 １ ５ 时起 北京市 空气 重 污染 黄色 预警 升级 为 橙色 预警 n 北京市 环保 监测中心 数据 显示 ４ 日 北京市 仍 受弱 气压 场 控制 风力 较弱 湿度 较大 回暖 明显 整体 大气 扩散 条件 不利 预计 空气质量 达到 ５ 级 重度 污染 ５ 日至 ７ 日 北京 受弱 东北 高压 影响 扩散 条件 改善 ７ 日 夜间 至 ８ 日受 较 强冷空气 影响 本次 污染 过程 结束 完'

In [486]:
def cut(content):
    def remove_extra_char(content):
        content =  [''.join(re.findall(re.compile('\w|\d'), con)) for con in content]
        return content
    
    content = remove_extra_char(content)
    
    jieba.enable_parallel(4)
    
    return [' '.join(jieba.cut(con)) for con in content]

In [487]:
tokens = cut(sample)
tokens[:5]

['新华社 北京 ４ 月 ３ 日电 记者 ３ 日 下午 从 北京市 环保局 获悉 ３ 日 １ ５ 时起 北京市 空气 重 污染 黄色 预警 升级 为 橙色 预警 n 北京市 环保 监测中心 数据 显示 ４ 日 北京市 仍 受弱 气压 场 控制 风力 较弱 湿度 较大 回暖 明显 整体 大气 扩散 条件 不利 预计 空气质量 达到 ５ 级 重度 污染 ５ 日至 ７ 日 北京 受弱 东北 高压 影响 扩散 条件 改善 ７ 日 夜间 至 ８ 日受 较 强冷空气 影响 本次 污染 过程 结束 完',
 '新华社 台北 5 月 26 日电 记者 章利新 马玉洁 台湾 女作家 林奕含 自杀 案 引发 岛内 各界 挞伐 狼师 呼吁 实施 补习班 老师 实名制 台 立法机构 26 日三读 修正 通过 补习 及 进修 教育 相关 规定 条文 增订 狼师 条款 规定 未来 补习班 负责人 与 员工 老师 执行 业务 或 对外 招生 时应 揭露 真实 姓名 n 新 通过 的 条文 规定 补习班 对外 进行 招生 书面 契约 广告 或 宣传 涉及 负责人 教职员工 姓名 时除 立案 名称 外 都 应 揭露 其 真实 姓名 n 条文 明确规定 短期 补习班 的 教职员工 若 有性 侵害 性骚扰 性 剥削 经 判刑 确定 或 通缉 在案 者 以及 有性 侵害 行为 或 情节 重大 的 性骚扰 性霸凌 损害 儿少 权益 行为 经 当地 教育 主管部门 查证 属实 者 应予 解聘 或 解雇 负责人 如果 有 涉及 性骚扰 性霸 凌性 侵害 等 情节 当地 教育 主管部门 应 废止 补习班 立案 n 条文 规定 补习班 聘雇 教职员工 前应 向 当地 教育 主管部门 查询 员工 有没有 性骚扰 等 记录 补习班 人员 知道 负责人 及 教职员工 对 学生 有性 侵害 等 行为 时除 依规 通报 外 也 应 通知 当地 教育 主管部门 若 补习班 未依 规定 通报 可处 负责人 新台币 5 万元 以上 25 万元 以下 罚款 并 按次 处罚 必要 时 并 令 其 停止 招生 或 废止 补习班 立案 n 根据 条文 外籍 教师 必须 缴交 所 持 护照 国家 的 行为 良好 证明 否则 不能 被 聘用 另外 台 卫生 福利 管理 部门 应 建立 曾犯 性骚扰 性 侵害 等 罪者 的 数据库 并 协助 各

In [488]:
with open('baidu_stop_word.txt', 'r') as f:
    baidu_stop_words = f.read()
baidu_stop_words = baidu_stop_words.split()

In [489]:
vectorizer = TfidfVectorizer(stop_words = baidu_stop_words)
X = vectorizer.fit_transform(tokens)
X.shape

/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(5000, 69533)

In [490]:
#vectorizer.vocabulary_

In [491]:
tf_idf_vector = X.toarray()

In [492]:
tf_idf_vector.shape

(5000, 69533)

In [493]:
vectorizer.vocabulary_["这是"]

62360

In [494]:
np.where(tf_idf_vector[0])

(array([ 4850,  5054,  5663, 16076, 16101, 16595, 18457, 21179, 23529,
        23831, 30478, 30750, 33193, 34890, 35688, 36223, 36310, 36768,
        37499, 37629, 37662, 37926, 38259, 39417, 40159, 41740, 42748,
        43209, 45457, 47554, 47556, 49269, 51288, 51291, 53282, 56911,
        58850, 61692, 61696, 62038, 64064, 67334, 67336, 67513, 68489,
        69188]),)

In [495]:
pca =  PCA(n_components = 100)
#pca_train_x = tf_idf_vector - np.mean(tf_idf_vector)
pca_train_x = pca.fit_transform(tf_idf_vector)
pca_train_x.shape

(5000, 100)

In [496]:
X_train, X_test, y_train, y_test = train_test_split(pca_train_x, y, test_size=0.33, random_state=42)

In [497]:
def performance(y, y_hat):
    print("precision: {}".format(precision_score(y, y_hat)))
    print("recall: {}".format(recall_score(y, y_hat)))
    print("roc_auc: {}".format(roc_auc_score(y, y_hat)))

In [498]:
class KNNCluster:
    def __init__(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y
        
    def run(self, x):
        x = x.reshape(1, -1)
        idx = np.argmax(cosine_similarity(self.train_x, x))
        return self.train_y[idx]
    
    def predict(self, test):
        y_hat = []
        for x in test:
            y = self.run(x)
            y_hat.append(y)
        
        return np.array(y_hat)
    
    def get_cos(self, v1, v2):
        return cosine(v1, v2)

In [499]:
knn = KNNCluster(X_train, y_train)

In [500]:
knn_y_hat = knn.predict(X_test)

In [501]:
performance(y_test, knn_y_hat)

precision: 0.948959032907992
recall: 0.942
roc_auc: 0.7176666666666667


In [502]:
from sklearn.neighbors import KNeighborsClassifier

In [503]:
neigh = KNeighborsClassifier(n_neighbors=3)

In [504]:
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [505]:
sk_knn_y_hat = neigh.predict(X_test)

In [506]:
performance(y_test, sk_knn_y_hat)

precision: 0.948959032907992
recall: 0.942
roc_auc: 0.7176666666666667


In [43]:
full_X_train, full_X_test, full_y_train, full_y_test = train_test_split(tf_idf_vector, y, test_size=0.33, random_state=42)

## SVM

In [507]:
from sklearn.svm import SVC

In [508]:
clf = SVC(kernel = 'rbf')

In [509]:
clf.fit(X_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [510]:
svm_y_hat = clf.predict(X_test)

In [511]:
performance(y_test, svm_y_hat)

precision: 0.9090909090909091
recall: 1.0
roc_auc: 0.5


## Bayes

In [512]:
from sklearn.naive_bayes import GaussianNB

In [513]:
clf = GaussianNB()

In [514]:
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [515]:
bayes_y_hat = clf.predict(X_test)

In [516]:
performance(y_test, bayes_y_hat)

precision: 0.9742319127849356
recall: 0.6553333333333333
roc_auc: 0.741


In [517]:
from scipy.stats import norm
from sklearn import preprocessing

In [518]:
classifier = set(y_train)

In [520]:
class BayesClassifier:
    def __init__(self, train_x, train_y):
        self.train_x = train_x
        self.train_y = train_y
        
        self.prepare()
        
    def prepare(self):
        self.classifier = list(set(self.train_y))
        
        self.mean = [np.mean(self.train_x[self.train_y==cls, :], axis=0) for cls in self.classifier]
        self.std = [np.std(self.train_x[self.train_y==cls, :], axis=0) for cls in self.classifier]
        self.mean = np.array(self.mean)
        self.std = np.array(self.std)
    
        self.p_classifier = [len(self.train_x[self.train_y==cls]) / len(self.train_x) for cls in self.classifier]
        
    def likehood(self, x, cls_idx):
        mean = self.mean[cls_idx,:]
        std = self.std[cls_idx,:]
        p_cls = self.p_classifier[cls_idx]
        
        joint = -np.sum(np.log(np.sqrt(2 * np.pi) * std)) - \
                    np.sum(np.power(x - mean, 2) / (2 * np.power(std, 2)))

        return joint + np.log(p_cls)
    
    def predict(self, test):
        y_hat = []
        
        for vector in test: 
            p_y = [self.likehood(vector, i) for i in range(len(self.classifier))]
        
            y_hat.append(self.classifier[np.argmax(p_y)])
                
        return np.array(y_hat)

In [521]:
bayes = BayesClassifier(X_train, y_train)

In [522]:
bayes_y_hat = bayes.predict(X_test)
sum(bayes_y_hat)

1009

In [524]:
performance(y_test, bayes_y_hat)

precision: 0.9742319127849356
recall: 0.6553333333333333
roc_auc: 0.741


## XGBoost

In [526]:
import xgboost as xgb

In [560]:
param_dist = {'objective':'binary:logistic', 'n_estimators':2}

In [566]:
clf = xgb.XGBModel(**param_dist)

In [571]:
clf.fit(X_train, y_train,
        eval_metric='auc')

XGBModel(base_score=0.5, booster='gbtree', colsample_bylevel=1,
     colsample_bynode=1, colsample_bytree=1, gamma=0,
     importance_type='gain', learning_rate=0.1, max_delta_step=0,
     max_depth=3, min_child_weight=1, missing=None, n_estimators=2,
     n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
     reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
     subsample=1, verbosity=1)

In [616]:
xgboost_y_hat = clf.predict(X_test)
xgboost_y_hat = list(map(lambda y : int(y > 0.56), xgboost_y_hat))

In [615]:
performance(y_test, xgboost_y_hat)

precision: 0.9762470308788599
recall: 0.822
roc_auc: 0.8110000000000002
